In [3]:
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import os

In [43]:
class NoiseAnalysis:
    def __init__(self, filename, input_path, output_path):
        self.output_path = output_path
        self.filename = filename
        self.dt = 0.1
        self.df = 50
        if input_path.endswith(('.wav')):
            self.rate, self.data = wavfile.read(input_path)

    def plot_spectrogram(self, filename, data):
        freq, time, spect= signal.spectrogram(data, self.rate, scaling='spectrum', window='hann')
        log_spec = np.log10(spect)
        plt.pcolormesh(time, freq, log_spec, shading='gouraud')
        plt.savefig(self.output_path+filename+"_spectr.png", dpi = 300)
        plt.clf()

        energy = np.sum(spect, axis=0)
        peaks = []
        t_i = int(self.dt * len(time) / (time[-1] - time[0]))
        freq_i = int(self.df * len(freq) / (freq[-1] - freq[0]))

        for i, _ in enumerate(freq):
            for j, _ in enumerate(time):
                t_min = max(0, j - t_i)
                t_max = min(len(time), j + t_i + 1)
                freq_min = max(0, i - freq_i)
                freq_max = min(len(freq), i + freq_i + 1)

                is_peak = True 

                for a in range(freq_min, freq_max):
                    for b in range(t_min, t_max):
                        if (a != i or b != j) and spect[i, j] < spect[a, b]:
                            is_peak = False
                            break
                    if not is_peak:
                        break

                if is_peak:
                    peaks.append(j)

        plt.plot(energy)
        plt.plot(peaks, energy[peaks], "*")
        plt.savefig(self.output_path+filename+"_peaks.png", dpi = 300)
        plt.clf()

    def plot_spectrogram_data(self):
        self.plot_spectrogram(self.filename, self.data)

    def butter_lowpass(self,cutoff, fs, order=5):
        n = 0.5 * fs
        normal_cutoff = cutoff / n
        b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
        return b, a
    
    def lowpass_filter(self,data, cutoff, fs, order=5):
        b, a = self.butter_lowpass(cutoff, fs, order=order)
        y = signal.filtfilt(b, a, data)
        return y
    
    def reduce_noise(self, cutoff = 3000):
        dd = signal.savgol_filter(self.data, window_length=101, polyorder=2)
        self.plot_spectrogram(self.filename+'_savgol', dd)
        wavfile.write(self.output_path+self.filename+'_savgol'+'.wav', self.rate, np.int16(dd))
        dd = self.lowpass_filter(self.data, cutoff, self.rate)
        self.plot_spectrogram(self.filename+'_lowpass', dd)
        wavfile.write(self.output_path+self.filename+'_lowpass'+'.wav', self.rate, np.int16(dd))
    

In [44]:
input_path = "./input/Piano.wav"
output_path = "./output/"
filename = "piano"

agent = NoiseAnalysis(filename, input_path, output_path)
agent.plot_spectrogram_data()
agent.reduce_noise()

[0, 45, 73, 144, 170, 221, 285, 334, 373, 432, 475, 504, 555, 588, 618, 657, 730, 793, 843, 869, 905, 941, 1060, 1109, 1132, 1187, 1233, 1312, 1341, 1400, 1428, 1453, 1494, 1530, 1559, 1607, 1647, 1736, 1770, 1808, 1884, 1916, 1968, 1990, 2033, 2095, 2120, 2153, 2215, 2242, 2344, 2370, 2397, 2426, 2451, 2477, 2504, 2529, 2583, 2637, 2671, 2710, 2758, 2888, 2932, 2981, 3065, 3100, 3154, 3208, 3240, 3278, 3324, 3355, 3412, 3463, 3492, 3518, 3584, 3626, 3713, 3811, 3876, 3899, 3996, 4019, 4086, 4114, 4153, 4192, 4257, 4282, 4351, 4379, 4441, 4480, 4584, 42, 96, 144, 170, 211, 285, 339, 373, 417, 460, 504, 674, 702, 793, 843, 869, 954, 1065, 1153, 1200, 1265, 1299, 1388, 1429, 1526, 1559, 1650, 1694, 1737, 1815, 1870, 1912, 1949, 2038, 2121, 2145, 2210, 2242, 2344, 2387, 2413, 2465, 2589, 2636, 2674, 2727, 2755, 2801, 2896, 2984, 3010, 3065, 3100, 3154, 3225, 3278, 3324, 3355, 3417, 3492, 3546, 3584, 3684, 3718, 3780, 3808, 3910, 3964, 4019, 4086, 4114, 4192, 4257, 4283, 4337, 4379, 4493, 

<Figure size 432x288 with 0 Axes>